In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import requests
from io import TextIOWrapper
from zipfile import ZipFile
import io

from matplotlib.colors import LinearSegmentedColormap

from numpy import array
from numpy import argmax

from sklearn import tree
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix


# Files were previously uploaded from data cleaning onto Github repository.
# Just need to pull them up with the following:

feat_url = "https://raw.githubusercontent.com/jasmultani5391/Census-Data/master/featDF.csv"
download1 = requests.get(feat_url).content

complete_url = "https://raw.githubusercontent.com/jasmultani5391/Census-Data/master/completeDF.csv"
download2 = requests.get(complete_url).content

# Read the downloaded content and turn it into a pandas dataframe
featDF = pd.read_csv(io.StringIO(download1.decode('utf-8')))
completeDF = pd.read_csv(io.StringIO(download2.decode('utf-8')))

#print(featDF.head(4))
#print(completeDF.head(4))



QUESTION: is changing random_state the best way to iterate?


In [2]:
# First, we must find the best K index that gives us the highest
# accuracy. We previously created a class called NearestK that contains
# the method to search for the best K value to use. We'll use the featDF
# over the completeDF because the former contains all the one-hot encoded
# columns that converted qualitative to quantitative data.

labelDF = featDF['salary_label']
featDF = featDF.drop(['salary_label'],
                     axis=1
                     )

In [5]:
trndfdata, tstdfdata, trndflbl, tstdflbl = train_test_split(featDF,
                                                            labelDF,
                                                            random_state=0)


rf_classifier = RandomForestClassifier(min_samples_leaf=70,
                                       n_estimators=150,
                                       bootstrap=True,
                                       oob_score=True, #use out-of-bag samples
                                       n_jobs=-1, 
                                       random_state=0,
                                       max_features='sqrt'
                                      )

# Fit classifier on train dataset.
rf_classifier.fit(trndfdata, np.ravel(trndflbl))

# Score based on train dataset.
train_df_score = rf_classifier.score(trndfdata, trndflbl)
print('Train Accuracy Score %: ' + str(round(train_df_score*100, 4)))

predict_trainlabels = rf_classifier.predict(trndfdata)
trainprecision = precision_score(trndflbl, predict_trainlabels, average='binary')
print('Train Precision %: ' + str(round(trainprecision*100, 4)))
            
trainrecall = recall_score(trndflbl, predict_trainlabels, average='binary')
print('Train Recall %: ' + str(round(trainrecall*100, 4)))

trainf1 = f1_score(trndflbl, predict_trainlabels, average='binary')
print('Train F1 %: ' + str(round(trainf1*100, 4)))


print('\n\n')
# Score based on test dataset.
test_df_score = rf_classifier.score(tstdfdata, tstdflbl)
print('Test Accuracy Score %: ' + str(round(test_df_score*100, 4)))

predict_testlabels = rf_classifier.predict(tstdfdata)
testprecision = precision_score(tstdflbl, predict_testlabels, average='binary')
print('Test Precision %: ' + str(round(testprecision*100, 4)))
            
testrecall = recall_score(tstdflbl, predict_testlabels, average='binary')
print('Test Recall %: ' + str(round(testrecall*100, 4)))

testf1 = f1_score(tstdflbl, predict_testlabels, average='binary')
print('Test F1 %: ' + str(round(testf1*100, 4)))


# Most important coefficients
important_coefficients = rf_classifier.feature_importances_
feature_columns = list(featDF.head(0))
importantcoef_columns = dict(zip(feature_columns, important_coefficients))
importantcoef_series = pd.Series(importantcoef_columns)
importantcoef_series = importantcoef_series.sort_values(ascending=False)
importantcoef = dict(importantcoef_series)
print(importantcoef_series)
# Confirm we have the total number of features included, which should be 45.
print(len(importantcoef))



Train Accuracy Score %: 85.5446
Train Precision %: 77.5862
Train Recall %: 52.6409
Train F1 %: 62.7244



Test Accuracy Score %: 84.5493
Test Precision %: 79.6407
Test Recall %: 51.9024
Test F1 %: 62.847
maritalstatus_ Married-civ-spouse       0.172178
relationship_ Husband                   0.156411
norm_edunum                             0.154003
norm_capitalgain                        0.148939
maritalstatus_ Never-married            0.056016
norm_age                                0.055552
norm_hoursperweek                       0.043771
occupation_ Exec-managerial             0.040258
occupation_ Prof-specialty              0.034098
norm_capitalloss                        0.026288
relationship_ Not-in-family             0.018556
relationship_ Wife                      0.017756
gender_label                            0.017497
relationship_ Own-child                 0.013268
maritalstatus_ Divorced                 0.009518
occupation_ Other-service               0.008812
relationship